<h1>Segmenting and clustering neighborhoods in Toronto.</h1>

Importing libs:

In [1]:
import urllib.request
import time
import  bs4 as bs
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)\

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


We will scrape wiki for list of postal codes of Canada using BeautifulSoup:

In [47]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
table_rows = []
table = soup.find('table')
table_rows = table.find_all('tr')

Converting scraped results to dataframe:

In [ ]:
l = []
for tr in table_rows[1:]:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

df = pd.DataFrame(l, columns=['PostalCode', 'Borough', 'Neighborhood'])

Replacing new line characters:

In [ ]:
df.replace(to_replace=r'(\r\n|\r|\n)', value=' ', regex=True, inplace=True)

If borough is not assigned - replace with NaN:

In [ ]:
df['Borough'].replace(to_replace=r'Not assigned', value=np.nan, regex=True, inplace=True)

Stripping white spaces from start and end of the line:

In [ ]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

Dropping NaNs:

In [ ]:
df.dropna(inplace=True)

Resetting index:

In [ ]:
df = df.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

If for some row Neighborhood is not assigned, but Borough is, assign Borough value to Neighborhood:

In [ ]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 0, df['Borough'], df['Neighborhood'])

Checking shape:

In [ ]:
df.shape

Since there is some inconsistency with Geocoder we will use prepared CSV file with Geospatial Coordinates

In [ ]:
data_geo = pd.read_csv("/Users/anastasiakulbaka/PycharmProjects/Test/Geospatial_Coordinates.csv")
df_geo = pd.DataFrame(data=data_geo)
df_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

Joining original Dataframe with Geospatial_Coordinates dataframe:

In [ ]:
cond = df['PostalCode'] == df_geo['PostalCode']
df = df.assign(Latitude = np.where(cond, df_geo['Latitude'].round(decimals=6), 'FALSE'))
df = df.assign(Longitude = np.where(cond, df_geo['Longitude'].round(decimals=6), 'FALSE'))

In [ ]:
df.to_csv('df_cleaned.csv')

In [ ]:
df.head()

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
from datetime import date
today = date.today()
# ddmmYY
VERSION = today.strftime("%Y%m%d")

In [ ]:
CLIENT_ID = 'I5XIAEZGJHJVXUPBQPUREOWFJ2C0KZBZXN2KK33HCHVI2QGY' # your Foursquare ID
CLIENT_SECRET = '5AWNJSIRQ1PGYICGJYC15E52DYJY2AF2MT0T1ACMZF5JOYDS' # your Foursquare Secret
#VERSION = '20200824' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Foursquare API version:' + VERSION)

radius = 500
LIMIT = 100
#url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
#url

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(16) # check the last columns!

In [ ]:
toronto_merged_n0_na = toronto_merged.dropna().reset_index()
toronto_merged_n0_na.astype({'Cluster Labels': 'int32'})

In [ ]:
#type(cluster)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_n0_na['Latitude'], toronto_merged_n0_na['Longitude'], toronto_merged_n0_na['Neighborhood'], toronto_merged_n0_na['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 0, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

In [ ]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 1, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

In [ ]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 2, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

In [ ]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 3, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]

In [ ]:
toronto_merged_n0_na.loc[toronto_merged_n0_na['Cluster Labels'] == 4, toronto_merged_n0_na.columns[[1] + list(range(5, toronto_merged_n0_na.shape[1]))]]